In [29]:
import torch
from torch.utils.data import Dataset
from src.data.negativesampler import NegativeSampler

import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--num_factors', type=int, default=15, help='Number of factors for FM')
parser.add_argument('--lr', type=float, default=0.005, help='Learning rate for fm training')
parser.add_argument('--weight_decay', type=float, default=0.1, help='Weight decay(for both FM and autoencoder)')
parser.add_argument('--num_epochs_ae', type=int, default=100,    help='Number of epochs')
parser.add_argument('--num_epochs_training', type=int, default=200,    help='Number of epochs')

parser.add_argument('--batch_size', type=int, default=1024, help='Batch size')
parser.add_argument('--ae_batch_size', type=int, default=256, help='Batch size for autoencoder')

parser.add_argument('--num_workers', type=int, default=20, help='Number of workers for dataloader')
parser.add_argument('--num_deep_layers', type=int, default=2, help='Number of deep layers')
parser.add_argument('--deep_layer_size', type=int, default=128, help='Size of deep layers')
parser.add_argument('--seed', type=int, default=42)
parser.add_argument('--save_model', type=bool, default=False)


parser.add_argument('--emb_dim', type=int, default=16, help='embedding dimension for DeepFM')
parser.add_argument('--num_embedding', type=int, default=200, help='Number of embedding for autoencoder') 
parser.add_argument('--embedding_type', type=str, default='original', help='AE or SVD or original')
parser.add_argument('--model_type', type=str, default='deepfm', help='fm or deepfm')
parser.add_argument('--topk', type=int, default=5, help='top k items to recommend')
parser.add_argument('--fold', type=int, default=1, help='fold number')
parser.add_argument('--isuniform', type=bool, default=True, help='isuniform')
parser.add_argument('--ratio_negative', type=int, default=0.2, help='ratio_negative')
parser.add_argument('--auto_lr', type=float, default=0.01, help='autoencoder learning rate')
parser.add_argument('--k', type=int, default=10, help='autoencoder k')
parser.add_argument('--num_eigenvector', type=int, default=10,help='Number of eigenvectors for SVD')
args = parser.parse_args("")




class CustomDataset(Dataset):
    def __init__(self,items,target,c):
        self.items=items
        self.target=target
        self.c=c
    def __len__(self):
        return self.items.shape[0]
    def __getitem__(self,idx):
        return self.items[idx],self.target[idx],self.c[idx]

from src.data.movielensdata import MovielensData
movielens=MovielensData('dataset/ml-100k','u.data',fold=1)
train,test=movielens.data_getter()
movie_info=movielens.movie_getter()
user_info=movielens.user_getter()
#merge train and movie_info
ns=NegativeSampler(args,train)
nssampled=ns.negativesample(args.isuniform)

Negative Sampling Started


100%|██████████| 943/943 [00:09<00:00, 102.75it/s]


        movie_id  user_id  user_frequency  movie_frequency  target  c
0            346        1             135              104       0  1
1            503        1             135               29       0  1
2           1647        1             135                1       0  1
3           1210        1             135               28       0  1
4            279        1             135               21       0  1
...          ...      ...             ...              ...     ... ..
294824       456      943             168               38       0  1
294825       994      943             168                6       0  1
294826      1282      943             168                8       0  1
294827       555      943             168                7       0  1
294828       803      943             168                7       0  1

[294829 rows x 6 columns]
Negative Sampling Finished


In [32]:
nssampled.columns
#label encoding
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(movie_info['movie_id'])
nssampled['movie_id']=le.transform(nssampled['movie_id'])

In [33]:
#decode
le.inverse_transform(nssampled['movie_id'])

array([   1,    2,    3, ..., 1282,  555,  803])

In [31]:

nssampled.merge(movie_info,on='movieId',how='left')
nssampled.merge(user_info,on='userId',how='left')

KeyError: 'movieId'